### Ranking Model 学习与使用

#### 环境查看

In [2]:
!pip list|grep -E 'torch|rechub'

torch                     2.6.0+cpu
torch-rechub              0.3.0


In [3]:
import torch_rechub

torch_rechub.__version__

'0.1.0'

#### 数据目录查看

In [4]:
!ls /data

criteo_sample.csv  ml-1m_sample.csv


#### 导入相关模块

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from torch_rechub.basic.features import DenseFeature, SparseFeature
from torch_rechub.utils.data import DataGenerator



In [7]:
# Load data
df = pd.read_csv("/data/criteo_sample.csv")

# Define feature columns
sparse_features = [f'C{i}' for i in range(1, 27)]
dense_features = [f'I{i}' for i in range(1, 14)]

#### 数据查看与验证

In [8]:
len(sparse_features)

26

In [9]:
len(dense_features)

13

In [11]:
df

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,NaN,32c7478e,0d4a6d1a,001f3601,92c878de
1,0,0.0,-1,63.0,40.0,1470.0,61.0,4.0,37.0,46.0,...,e5ba7672,d3303ea5,21ddcdc9,b1252a9d,7633c7c8,NaN,32c7478e,17f458f7,001f3601,71236095
2,0,0.0,370,4.0,1.0,1787.0,65.0,14.0,25.0,489.0,...,3486227d,642f2610,55dd3565,b1252a9d,5c8dc711,NaN,423fab69,45ab94c8,2bf691b1,c84c4aec
3,1,19.0,10,30.0,10.0,1.0,3.0,33.0,47.0,126.0,...,e5ba7672,a78bd508,21ddcdc9,5840adea,c2a93b37,NaN,32c7478e,1793a828,e8b83407,2fede552
4,0,0.0,0,36.0,22.0,4684.0,217.0,9.0,35.0,135.0,...,e5ba7672,7ce63c71,NaN,NaN,af5dc647,NaN,dbb486d7,1793a828,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0,NaN,0,1.0,33.0,11774.0,NaN,0.0,1.0,502.0,...,776ce399,f2fc99b1,NaN,NaN,38879cfe,ad3062eb,32c7478e,7836b4d5,NaN,NaN
111,0,NaN,1,14.0,3.0,3008.0,15.0,6.0,5.0,146.0,...,e5ba7672,1616f155,21ddcdc9,5840adea,67afd8d0,NaN,c7dc6720,e3aea32f,9b3e8820,e75c9ae9
112,1,0.0,1,27.0,38.0,1499.0,73.0,14.0,35.0,269.0,...,e5ba7672,6fc84bfb,NaN,NaN,4f1aa25f,NaN,423fab69,ded4aac9,NaN,NaN
113,0,NaN,5,44.0,4.0,12143.0,NaN,0.0,4.0,4.0,...,776ce399,582152eb,21ddcdc9,5840adea,d1d4f4a9,ad3062eb,3a171ecb,03955d00,001f3601,4e7af834


In [12]:
df.columns

Index(['label', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10',
       'I11', 'I12', 'I13', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
       'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18',
       'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26'],
      dtype='str')

#### 数据预处理

In [36]:
df[sparse_features] = df[sparse_features].fillna('0')
df[dense_features] = df[dense_features].fillna(0)

In [37]:
df

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,0,32c7478e,0d4a6d1a,001f3601,92c878de
1,0,0.0,-1,63.0,40.0,1470.0,61.0,4.0,37.0,46.0,...,e5ba7672,d3303ea5,21ddcdc9,b1252a9d,7633c7c8,0,32c7478e,17f458f7,001f3601,71236095
2,0,0.0,370,4.0,1.0,1787.0,65.0,14.0,25.0,489.0,...,3486227d,642f2610,55dd3565,b1252a9d,5c8dc711,0,423fab69,45ab94c8,2bf691b1,c84c4aec
3,1,19.0,10,30.0,10.0,1.0,3.0,33.0,47.0,126.0,...,e5ba7672,a78bd508,21ddcdc9,5840adea,c2a93b37,0,32c7478e,1793a828,e8b83407,2fede552
4,0,0.0,0,36.0,22.0,4684.0,217.0,9.0,35.0,135.0,...,e5ba7672,7ce63c71,0,0,af5dc647,0,dbb486d7,1793a828,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0,0.0,0,1.0,33.0,11774.0,0.0,0.0,1.0,502.0,...,776ce399,f2fc99b1,0,0,38879cfe,ad3062eb,32c7478e,7836b4d5,0,0
111,0,0.0,1,14.0,3.0,3008.0,15.0,6.0,5.0,146.0,...,e5ba7672,1616f155,21ddcdc9,5840adea,67afd8d0,0,c7dc6720,e3aea32f,9b3e8820,e75c9ae9
112,1,0.0,1,27.0,38.0,1499.0,73.0,14.0,35.0,269.0,...,e5ba7672,6fc84bfb,0,0,4f1aa25f,0,423fab69,ded4aac9,0,0
113,0,0.0,5,44.0,4.0,12143.0,0.0,0.0,4.0,4.0,...,776ce399,582152eb,21ddcdc9,5840adea,d1d4f4a9,ad3062eb,3a171ecb,03955d00,001f3601,4e7af834


In [38]:
def convert_numeric_feature(val):
    v = int(val)
    if v > 2:
        return int(np.log(v)**2)
    else:
        return v - 2

In [39]:
for feat in dense_features:  # discretize dense feature and as new sparse feature
    sparse_features.append(feat + "_cat")
    df[feat + "_cat"] = df[feat].apply(lambda x: convert_numeric_feature(x))

#### 数据缩放

In [40]:
sca = MinMaxScaler()  # scaler dense feature
df[dense_features] = sca.fit_transform(df[dense_features])

In [41]:
df

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,I4_cat,I5_cat,I6_cat,I7_cat,I8_cat,I9_cat,I10_cat,I11_cat,I12_cat,I13_cat
0,0,0.000000,0.000349,0.036945,0.310345,0.003922,0.137464,0.023256,0.653061,0.046077,...,10,57,24,1,12,13,-2,-1,-2,10
1,0,0.000000,0.000000,0.022380,0.459770,0.002898,0.059051,0.023256,0.755102,0.057285,...,13,53,16,1,13,14,-2,-1,-2,13
2,0,0.000000,0.129449,0.001421,0.011494,0.003522,0.062924,0.081395,0.510204,0.608966,...,-1,56,17,6,10,38,-2,3,-2,10
3,1,0.513514,0.003838,0.010657,0.114943,0.000002,0.002904,0.191860,0.959184,0.156912,...,5,-1,1,12,14,23,1,2,-2,0
4,0,0.000000,0.000349,0.012789,0.252874,0.009233,0.210068,0.052326,0.714286,0.168120,...,9,71,28,4,12,24,-2,-1,-2,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0,0.000000,0.000349,0.000355,0.379310,0.023208,0.000000,0.000000,0.020408,0.625156,...,12,87,-2,-2,-1,38,-2,-2,-2,12
111,0,0.000000,0.000698,0.004973,0.034483,0.005929,0.014521,0.034884,0.102041,0.181818,...,1,64,7,3,2,24,-2,1,-2,1
112,1,0.000000,0.000698,0.009591,0.436782,0.002955,0.070668,0.081395,0.714286,0.334994,...,13,53,18,6,12,31,-2,1,-2,13
113,0,0.000000,0.002094,0.015631,0.045977,0.023935,0.000000,0.000000,0.081633,0.004981,...,1,88,-2,-2,1,1,-2,-2,-2,1


In [42]:
for feat in sparse_features:  # encode sparse feature
    lbe = LabelEncoder()
    df[feat] = lbe.fit_transform(df[feat])

In [43]:
df

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,I4_cat,I5_cat,I6_cat,I7_cat,I8_cat,I9_cat,I10_cat,I11_cat,I12_cat,I13_cat
0,0,0.000000,0.000349,0.036945,0.310345,0.003922,0.137464,0.023256,0.653061,0.046077,...,12,34,24,3,14,15,0,1,0,12
1,0,0.000000,0.000000,0.022380,0.459770,0.002898,0.059051,0.023256,0.755102,0.057285,...,15,30,17,3,15,16,0,1,0,15
2,0,0.000000,0.129449,0.001421,0.011494,0.003522,0.062924,0.081395,0.510204,0.608966,...,1,33,18,8,12,34,0,5,0,12
3,1,0.513514,0.003838,0.010657,0.114943,0.000002,0.002904,0.191860,0.959184,0.156912,...,7,1,3,14,16,24,3,4,0,2
4,0,0.000000,0.000349,0.012789,0.252874,0.009233,0.210068,0.052326,0.714286,0.168120,...,11,43,27,6,14,25,0,1,0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0,0.000000,0.000349,0.000355,0.379310,0.023208,0.000000,0.000000,0.020408,0.625156,...,14,51,0,0,1,34,0,0,0,14
111,0,0.000000,0.000698,0.004973,0.034483,0.005929,0.014521,0.034884,0.102041,0.181818,...,3,38,9,5,4,25,0,3,0,3
112,1,0.000000,0.000698,0.009591,0.436782,0.002955,0.070668,0.081395,0.714286,0.334994,...,15,30,19,8,14,31,0,3,0,15
113,0,0.000000,0.002094,0.015631,0.045977,0.023935,0.000000,0.000000,0.081633,0.004981,...,3,52,0,0,3,3,0,0,0,3


In [ ]:
# # Define features
# dense_feas = [DenseFeature(name) for name in dense_features]
# sparse_feas = [SparseFeature(name, vocab_size=df[name].nunique(), embed_dim=16) for name in sparse_features]


In [45]:
dense_feas = [DenseFeature(feature_name) for feature_name in dense_features]
sparse_feas = [SparseFeature(feature_name, vocab_size=df[feature_name].nunique(), embed_dim=16) for feature_name in sparse_features]
ffm_linear_feas = [SparseFeature(feature.name, vocab_size=feature.vocab_size, embed_dim=1) for feature in sparse_feas]
ffm_cross_feas = [SparseFeature(feature.name, vocab_size=feature.vocab_size * len(sparse_feas), embed_dim=10) for feature in sparse_feas]
y = df["label"]
del df["label"]
x = df

In [46]:
x

,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,...,I4_cat,I5_cat,I6_cat,I7_cat,I8_cat,I9_cat,I10_cat,I11_cat,I12_cat,I13_cat
0,0.000000,0.000349,0.036945,0.310345,0.003922,0.137464,0.023256,0.653061,0.046077,0.000000,...,12,34,24,3,14,15,0,1,0,12
1,0.000000,0.000000,0.022380,0.459770,0.002898,0.059051,0.023256,0.755102,0.057285,0.000000,...,15,30,17,3,15,16,0,1,0,15
2,0.000000,0.129449,0.001421,0.011494,0.003522,0.062924,0.081395,0.510204,0.608966,0.000000,...,1,33,18,8,12,34,0,5,0,12
3,0.513514,0.003838,0.010657,0.114943,0.000002,0.002904,0.191860,0.959184,0.156912,1.000000,...,7,1,3,14,16,24,3,4,0,2
4,0.000000,0.000349,0.012789,0.252874,0.009233,0.210068,0.052326,0.714286,0.168120,0.000000,...,11,43,27,6,14,25,0,1,0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0.000000,0.000349,0.000355,0.379310,0.023208,0.000000,0.000000,0.020408,0.625156,0.000000,...,14,51,0,0,1,34,0,0,0,14
111,0.000000,0.000698,0.004973,0.034483,0.005929,0.014521,0.034884,0.102041,0.181818,0.000000,...,3,38,9,5,4,25,0,3,0,3
112,0.000000,0.000698,0.009591,0.436782,0.002955,0.070668,0.081395,0.714286,0.334994,0.000000,...,15,30,19,8,14,31,0,3,0,15
113,0.000000,0.002094,0.015631,0.045977,0.023935,0.000000,0.000000,0.081633,0.004981,0.000000,...,3,52,0,0,3,3,0,0,0,3


In [47]:
y

0      0
1      0
2      0
3      1
4      0
      ..
110    0
111    0
112    1
113    0
114    1
Name: label, Length: 115, dtype: int64

In [100]:
from torch_rechub.utils.data import DataGenerator


dg = DataGenerator(x, y)
train_dl, val_dl, test_dl = dg.generate_dataloader(split_ratio=[0.7, 0.1], batch_size=256)



the samples of train : val : test are  80 : 11 : 24


#### 久闻W&D模型, 今天见识一下

In [15]:
# Wide & Deep Model
# Wide & Deep model combines memorization and generalization capabilities:


from torch_rechub.models.ranking import WideDeep
from torch_rechub.trainers import CTRTrainer

In [74]:
# Model configuration
model = WideDeep(
    wide_features=dense_feas,
    deep_features=sparse_feas,
    mlp_params={"dims": [256, 128, 64], "dropout": 0.2, "activation": "relu"}
)


#### 没有GPU, 先用CPU对付一下~~

In [75]:
# Training configuration
trainer = CTRTrainer(
    model=model,
    optimizer_params={"lr": 0.001, "weight_decay": 1e-5},
    n_epoch=10,
    device="cpu"
)

### 开训

In [76]:
# Train model
trainer.fit(train_dl, val_dl)

epoch: 0


validation: 100%|██████████| 1/1 [00:00<00:00, 37.77it/s]


epoch: 0 validation: auc: 0.5357142857142857
epoch: 1


validation: 100%|██████████| 1/1 [00:00<00:00, 37.53it/s]


epoch: 1 validation: auc: 0.5357142857142857
epoch: 2


...

epoch: 7 validation: auc: 0.5357142857142857
epoch: 8


validation: 100%|██████████| 1/1 [00:00<00:00, 31.93it/s]


epoch: 8 validation: auc: 0.5357142857142857
epoch: 9


validation: 100%|██████████| 1/1 [00:00<00:00, 40.03it/s]

epoch: 9 validation: auc: 0.5357142857142857


In [77]:
auc = trainer.evaluate(trainer.model, test_dl)

validation: 100%|██████████| 1/1 [00:00<00:00, 21.41it/s]


In [78]:
auc

0.5078125

In [60]:
len(df)

115

In [66]:
list(train_dl)[0][-1].shape

torch.Size([80])

In [67]:
list(val_dl)[0][-1].shape

torch.Size([11])

In [68]:
list(test_dl)[0][-1].shape

torch.Size([24])

#### 效果很差啊

结果分析:

1. 数据太小, 一共只有115条记录, taining集只有70%*115 = 80条?
2. 模型参数选择不合适?
3. 是否W&D不合适这个场景数据?

#### 调整参数

In [108]:
def make_trials(dims=[36, 128, 64], lr=1e-3, n_epoch=10, activation='relu'):
    model = WideDeep(
        wide_features=dense_feas,
        deep_features=sparse_feas,
        mlp_params={"dims": dims, "dropout": 0.2, "activation": activation}
    )
    # Training configuration
    trainer = CTRTrainer(
        model=model,
        optimizer_params={"lr": lr, "weight_decay": 1e-5},
        n_epoch=n_epoch,
        device="cpu"
    )

    trainer.fit(train_dl, val_dl)
    auc = trainer.evaluate(trainer.model, test_dl)

    print(f'{auc=}')

    return model, trainer, auc

    

In [84]:
model1, trainer1, auc1 = make_trials()

epoch: 0


validation: 100%|██████████| 1/1 [00:00<00:00, 34.35it/s]


epoch: 0 validation: auc: 1.0
epoch: 1


validation: 100%|██████████| 1/1 [00:00<00:00, 38.39it/s]


epoch: 1 validation: auc: 1.0
epoch: 2


...

epoch: 8 validation: auc: 1.0
epoch: 9


validation: 100%|██████████| 1/1 [00:00<00:00, 39.12it/s]


epoch: 9 validation: auc: 1.0


validation: 100%|██████████| 1/1 [00:00<00:00, 20.00it/s]

auc=0.75


In [85]:
model1, trainer1, auc1 = make_trials()

epoch: 0


validation: 100%|██████████| 1/1 [00:00<00:00, 36.90it/s]


epoch: 0 validation: auc: 0.75
epoch: 1


validation: 100%|██████████| 1/1 [00:00<00:00, 24.46it/s]


epoch: 1 validation: auc: 0.75
epoch: 2


...

epoch: 8 validation: auc: 0.7142857142857142
epoch: 9


validation: 100%|██████████| 1/1 [00:00<00:00, 41.73it/s]


epoch: 9 validation: auc: 0.7142857142857142


validation: 100%|██████████| 1/1 [00:00<00:00, 22.97it/s]

auc=0.8125


In [86]:
model1, trainer1, auc1 = make_trials()

epoch: 0


validation: 100%|██████████| 1/1 [00:00<00:00, 39.68it/s]


epoch: 0 validation: auc: 0.5357142857142857
epoch: 1


validation: 100%|██████████| 1/1 [00:00<00:00, 37.24it/s]


epoch: 1 validation: auc: 0.5357142857142857
epoch: 2


...

epoch: 8 validation: auc: 0.5357142857142857
epoch: 9


validation: 100%|██████████| 1/1 [00:00<00:00, 29.42it/s]


epoch: 9 validation: auc: 0.5357142857142857


validation: 100%|██████████| 1/1 [00:00<00:00, 13.29it/s]

auc=0.2734375


In [88]:
import torch
torch.manual_seed(2022)

In [89]:
model1, trainer1, auc1 = make_trials()

epoch: 0


validation: 100%|██████████| 1/1 [00:00<00:00, 23.72it/s]


epoch: 0 validation: auc: 0.8214285714285714
epoch: 1


validation: 100%|██████████| 1/1 [00:00<00:00, 34.18it/s]


epoch: 1 validation: auc: 0.8214285714285714
epoch: 2


...

epoch: 8 validation: auc: 0.8214285714285714
epoch: 9


validation: 100%|██████████| 1/1 [00:00<00:00, 26.45it/s]


epoch: 9 validation: auc: 0.8214285714285714


validation: 100%|██████████| 1/1 [00:00<00:00, 21.62it/s]

auc=0.796875


In [90]:
model1, trainer1, auc1 = make_trials()

epoch: 0


validation: 100%|██████████| 1/1 [00:00<00:00, 35.32it/s]


epoch: 0 validation: auc: 0.7142857142857143
epoch: 1


validation: 100%|██████████| 1/1 [00:00<00:00, 38.26it/s]


epoch: 1 validation: auc: 0.7142857142857143
epoch: 2


...

epoch: 8 validation: auc: 0.7142857142857143
epoch: 9


validation: 100%|██████████| 1/1 [00:00<00:00, 18.26it/s]


epoch: 9 validation: auc: 0.7142857142857143


validation: 100%|██████████| 1/1 [00:00<00:00, 11.19it/s]


auc=0.296875


In [93]:
import random

def setup_seed(seed):
    
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True

In [94]:
setup_seed(2022)

In [95]:
model1, trainer1, auc1 = make_trials()

epoch: 0


validation: 100%|██████████| 1/1 [00:00<00:00, 35.64it/s]


epoch: 0 validation: auc: 0.8214285714285714
epoch: 1


validation: 100%|██████████| 1/1 [00:00<00:00, 37.77it/s]


epoch: 1 validation: auc: 0.8214285714285714
epoch: 2


...

epoch: 8 validation: auc: 0.8214285714285714
epoch: 9


validation: 100%|██████████| 1/1 [00:00<00:00, 26.09it/s]


epoch: 9 validation: auc: 0.8214285714285714


validation: 100%|██████████| 1/1 [00:00<00:00, 17.08it/s]

auc=0.796875


In [96]:
model1, trainer1, auc1 = make_trials()

epoch: 0


validation: 100%|██████████| 1/1 [00:00<00:00, 40.01it/s]

epoch: 0 validation: auc: 0.7142857142857143
epoch: 1



validation: 100%|██████████| 1/1 [00:00<00:00, 33.60it/s]


epoch: 1 validation: auc: 0.7142857142857143
epoch: 2


...

epoch: 8 validation: auc: 0.7142857142857143
epoch: 9


validation: 100%|██████████| 1/1 [00:00<00:00, 36.76it/s]


epoch: 9 validation: auc: 0.7142857142857143


validation: 100%|██████████| 1/1 [00:00<00:00, 19.24it/s]

auc=0.2890625


In [97]:
model1, trainer1, auc1 = make_trials()

epoch: 0


validation: 100%|██████████| 1/1 [00:00<00:00, 30.18it/s]


epoch: 0 validation: auc: 0.6428571428571429
epoch: 1


validation: 100%|██████████| 1/1 [00:00<00:00, 41.78it/s]


epoch: 1 validation: auc: 0.6428571428571429
epoch: 2


...

epoch: 8 validation: auc: 0.6428571428571429
epoch: 9


validation: 100%|██████████| 1/1 [00:00<00:00, 35.82it/s]


epoch: 9 validation: auc: 0.6428571428571429


validation: 100%|██████████| 1/1 [00:00<00:00, 21.04it/s]

auc=0.296875


In [98]:
model1, trainer1, auc1 = make_trials()

epoch: 0


validation: 100%|██████████| 1/1 [00:00<00:00, 26.08it/s]


epoch: 0 validation: auc: 0.6071428571428571
epoch: 1


validation: 100%|██████████| 1/1 [00:00<00:00, 37.74it/s]


epoch: 1 validation: auc: 0.6428571428571429
epoch: 2


...

epoch: 8 validation: auc: 0.6785714285714286
epoch: 9


validation: 100%|██████████| 1/1 [00:00<00:00, 41.82it/s]


epoch: 9 validation: auc: 0.6785714285714286


validation: 100%|██████████| 1/1 [00:00<00:00, 22.99it/s]

auc=0.359375


In [104]:
### 固定worker数量=1
dg = DataGenerator(x, y)
train_dl, val_dl, test_dl = dg.generate_dataloader(split_ratio=[0.7, 0.1], batch_size=256, num_workers=2)

the samples of train : val : test are  80 : 11 : 24


In [105]:
model1, trainer1, auc1 = make_trials()

epoch: 0


validation: 100%|██████████| 1/1 [00:00<00:00,  7.55it/s]


epoch: 0 validation: auc: 0.8214285714285714
epoch: 1


validation: 100%|██████████| 1/1 [00:00<00:00,  7.45it/s]


epoch: 1 validation: auc: 0.8214285714285714
epoch: 2


...

epoch: 8 validation: auc: 0.8214285714285714
epoch: 9


validation: 100%|██████████| 1/1 [00:00<00:00,  7.19it/s]


epoch: 9 validation: auc: 0.8214285714285714


validation: 100%|██████████| 1/1 [00:00<00:00,  6.22it/s]

auc=0.9125


In [106]:
model1, trainer1, auc1 = make_trials()

epoch: 0


validation: 100%|██████████| 1/1 [00:00<00:00,  7.15it/s]


epoch: 0 validation: auc: 0.4642857142857143
epoch: 1


validation: 100%|██████████| 1/1 [00:00<00:00,  7.20it/s]


epoch: 1 validation: auc: 0.4642857142857143
epoch: 2


...

epoch: 8 validation: auc: 0.4642857142857143
epoch: 9


validation: 100%|██████████| 1/1 [00:00<00:00,  6.76it/s]


epoch: 9 validation: auc: 0.4642857142857143


validation: 100%|██████████| 1/1 [00:00<00:00,  5.12it/s]


auc=0.2875


In [109]:
def make_trials(train_dl, val_dl, test_dl, dims=[36, 128, 64], lr=1e-3, n_epoch=10, activation='relu'):
    model = WideDeep(
        wide_features=dense_feas,
        deep_features=sparse_feas,
        mlp_params={"dims": dims, "dropout": 0.2, "activation": activation}
    )
    # Training configuration
    trainer = CTRTrainer(
        model=model,
        optimizer_params={"lr": lr, "weight_decay": 1e-5},
        n_epoch=n_epoch,
        device="cpu"
    )

    trainer.fit(train_dl, val_dl)
    auc = trainer.evaluate(trainer.model, test_dl)

    print(f'{auc=}')

    return model, trainer, auc

In [110]:
model1, trainer1, auc1 = make_trials(train_dl, val_dl, test_dl)

epoch: 0


validation: 100%|██████████| 1/1 [00:00<00:00,  6.12it/s]


epoch: 0 validation: auc: 0.3214285714285714
epoch: 1


validation: 100%|██████████| 1/1 [00:00<00:00,  6.14it/s]


epoch: 1 validation: auc: 0.2857142857142857
epoch: 2


...

epoch: 8 validation: auc: 0.3214285714285714
epoch: 9


validation: 100%|██████████| 1/1 [00:00<00:00,  7.45it/s]


epoch: 9 validation: auc: 0.25


validation: 100%|██████████| 1/1 [00:00<00:00,  3.22it/s]

auc=0.7250000000000001


#### 调参不易, 随机种子需要再进一步固化